
# 🤖 Agentic Memory Redaction in Multi-Agent Systems

This notebook demonstrates how to manage access to sensitive information in a multi-agent system. 

Inspired by concepts from the [MemGPT paper](https://arxiv.org/pdf/2310.08560), we will 
- create a system where an "Admin" agent can view confidential data, 
- while a "Junior" agent receives a redacted version. 

This is crucial for building secure and compliant AI systems.


In [1]:
# -*- coding: utf-8 -*-
"""
# 🤖 Agentic Memory Redaction in Multi-Agent Systems

This notebook demonstrates how to manage access to sensitive information in a multi-agent system. Inspired by concepts from the [MemGPT paper](https://arxiv.org/pdf/2310.08560), we will create a system where an "Admin" agent can view confidential data, while a "Junior" agent receives a redacted version. This is crucial for building secure and compliant AI systems.
"""

#@title ## 1. Setup and Configuration
# Install necessary libraries
#!pip install -q google-generativeai fpdf PyPDF2

import google.generativeai as genai
import json
import re
from fpdf import FPDF
import PyPDF2
# Import the function from your helper file
from helper import get_gemini_api_key

# --- Configuration ---
# Note: To run this, you'll need a helper.py file with a get_gemini_key() function.
try:
    api_key = get_gemini_api_key()
    if not api_key:
        raise ValueError("API key not found in helper.py")
    genai.configure(api_key=api_key)
    print("✅ Gemini API configured successfully!")
except Exception as e:
    print(f"🚨 Error configuring API key: {e}")
    print("Please ensure your helper.py file is in the same directory and returns a valid API key.")

"""---
## Concept: The Redaction Controller 🔒

To manage data access, we'll create a `RedactionController`. This controller will act as a gatekeeper, checking an agent's security level before granting access to information. If the agent does not have admin rights, the controller will redact sensitive data before sharing it.
"""

#@title ### Example: The Redaction Controller and Agent Roles

# 1. Define the Redaction Controller
class RedactionController:
    def __init__(self, fields_to_redact):
        self.fields_to_redact = fields_to_redact
        # Simple regex for things that look like names, emails, and phones
        self.pii_pattern = re.compile(
            r'\b[A-Z][a-z]+ [A-Z][a-z]+\b|'  # Names like John Doe
            r'\b\d{3}-\d{3}-\d{4}\b|'      # Phone numbers like 555-123-4567
            r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b' # Emails
        )

    def redact(self, text):
        """Redacts specified fields and common PII from a block of text."""
        # Redact specific key-value fields
        for field in self.fields_to_redact:
            text = re.sub(f"{field}:.*", f"{field}: [REDACTED]", text, flags=re.IGNORECASE)
        # Redact common PII patterns
        text = self.pii_pattern.sub("[REDACTED]", text)
        return text

# 2. Define Agent Roles
hiring_manager_agent = {"role": "Hiring Manager", "admin": True}
junior_analyst_agent = {"role": "Junior Analyst", "admin": False}

# 3. Initialize the Controller
controller = RedactionController(fields_to_redact=["Name", "Phone", "Email"])

"""---
## Scenario 1: Redaction from External Memory (PDF)

Here, we'll simulate agents accessing a resume from an external PDF document. The Hiring Manager will see the full document, while the Junior Analyst will see a redacted version.
"""

#@title ### Example: Accessing a Resume PDF

# 1. Create a dummy PDF with sensitive information
resume_text = """
Name: Jane Doe
Phone: 555-867-5309
Email: jane.doe@email.com
---
Summary:
A highly motivated software engineer with 5 years of experience in Python and cloud computing.
"""
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.multi_cell(0, 10, txt=resume_text)
pdf.output("resume.pdf")
print("📄 resume.pdf created.")

# 2. Define the tool to access the document
def get_document(agent, document_path):
    """Accesses a document and applies redaction based on agent's security level."""
    # FIX: Use PyPDF2 to properly extract text from the PDF
    text_content = ""
    with open(document_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text_content += page.extract_text()

    if agent["admin"]:
        print(f"✅ {agent['role']} has admin access. Returning full document.")
        return text_content
    else:
        print(f"🔒 {agent['role']} has restricted access. Redacting document.")
        return controller.redact(text_content)

# 3. Simulate agents accessing the PDF
print("\n--- Hiring Manager's View ---")
manager_view = get_document(hiring_manager_agent, "resume.pdf")
print(manager_view)

print("\n--- Junior Analyst's View ---")
analyst_view = get_document(junior_analyst_agent, "resume.pdf")
print(analyst_view)


"""---
## Scenario 2: Redaction of Internal Memory

Now, let's simulate the redaction of internal, agent-to-agent communication. The Hiring Manager will have a sensitive note in its working memory, which will be redacted before being shared with the Junior Analyst.
"""

#@title ### Example: Sharing a Sensitive Internal Note

# 1. The Hiring Manager has a sensitive note in its internal memory.
hiring_manager_memory = {
    "candidate_note": "Candidate Jane Doe is a strong hire. Contact her at 555-867-5309 to schedule an interview. Offer a salary of $120,000."
}

# 2. Define the tool for sharing internal memory
def share_internal_note(requesting_agent, source_memory):
    """Shares an internal note, redacting based on security level."""
    note = source_memory["candidate_note"]
    if requesting_agent["admin"]:
         print(f"✅ {requesting_agent['role']} has admin access. Returning full note.")
         return note
    else:
        print(f"🔒 {requesting_agent['role']} has restricted access. Redacting note.")
        # We can add more specific redaction rules here if needed
        redacted_note = controller.redact(note)
        redacted_note = re.sub(r'\$\d{1,3}(,\d{3})*', "[REDACTED SALARY]", redacted_note)
        return redacted_note

# 3. Simulate the Junior Analyst requesting the note
print("\n--- Hiring Manager's Internal Note ---")
print(hiring_manager_memory["candidate_note"])

print("\n--- Junior Analyst's View of the Shared Note ---")
shared_note = share_internal_note(junior_analyst_agent, hiring_manager_memory)
print(shared_note)


✅ Gemini API configured successfully!
📄 resume.pdf created.

--- Hiring Manager's View ---
✅ Hiring Manager has admin access. Returning full document.
Name: Jane Doe
Phone: 555-867-5309
Email: jane.doe@email.com
---
Summary:
A highly motivated software engineer with 5 years of experience in Python and cloud computing.

--- Junior Analyst's View ---
🔒 Junior Analyst has restricted access. Redacting document.
Name: [REDACTED]
Phone: [REDACTED]
Email: [REDACTED]
---
Summary:
A highly motivated software engineer with 5 years of experience in Python and cloud computing.

--- Hiring Manager's Internal Note ---
Candidate Jane Doe is a strong hire. Contact her at 555-867-5309 to schedule an interview. Offer a salary of $120,000.

--- Junior Analyst's View of the Shared Note ---
🔒 Junior Analyst has restricted access. Redacting note.
[REDACTED] Doe is a strong hire. Contact her at [REDACTED] to schedule an interview. Offer a salary of [REDACTED SALARY].


/opt/anaconda3/envs/gl_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
